<a href="https://colab.research.google.com/github/AKhilRaghav0/Linkedin_Learning_Ripper/blob/main/Linkedin_rip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Requirements
!pip install llvd ipyfilechooser
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title [3] Cookie Creator / OR Upload over the next cell.

import os
from ipywidgets import widgets

li_at_widget = widgets.Text(
    description='li_at:',
    placeholder='Enter li_at value',
    layout={'width': '50%'}
)

jsessionid_widget = widgets.Text(
    description='JSESSIONID:',
    placeholder='Enter JSESSIONID value',
    layout={'width': '50%'}
)

display(li_at_widget)
display(jsessionid_widget)

def on_button_click(b):
    li_at = li_at_widget.value.strip()
    jsessionid = jsessionid_widget.value.strip()

    with open('cookies.txt', 'w') as f:
        f.write(f'li_at={li_at}\n')
        f.write(f'JSESSIONID="{jsessionid}"\n')

button = widgets.Button(description='Save')
button.on_click(on_button_click)
display(button)


In [ ]:
#@title Cookies Upload
from google.colab import files

uploaded = files.upload()

if 'cookies.txt' in uploaded:
    print('Cookies file uploaded successfully!')
else:
    print('Error uploading cookies file.')


In [ ]:
#@title Course Downloader
import ipywidgets as widgets

# Function to download course
def download_course(course_slug, resolution, download_location, download_subtitle):
    res = resolution[0:3]
    if download_subtitle:
        !llvd -c "$course_slug" -r "$res" --cookies --caption
    else:
        !llvd -c "$course_slug" -r "$res" --cookies
    !cp -r "$course_slug" "$download_location"

# Create widgets for inputs
course_slug_widget = widgets.Text(
    value='',
    placeholder='Enter course slug...',
    description='Course Slug:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

resolution_widget = widgets.Dropdown(
    options=['720p', '540p', '360p'],
    value='720p',
    description='Resolution:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

download_location_widget = widgets.Text(
    value='/content/',
    placeholder='Enter download location...',
    description='Download Location:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

download_subtitle_widget = widgets.Checkbox(
    value=True,
    description='Download Subtitles',
    style={'description_width': 'initial'}
)

# Create button to initiate download
download_button = widgets.Button(
    description='Download',
    button_style='success',
    layout=widgets.Layout(width='25%')
)

# Function to handle button click event
def on_download_button_click(b):
    course_slug = course_slug_widget.value
    resolution = resolution_widget.value
    download_location = download_location_widget.value
    download_subtitle = download_subtitle_widget.value
    download_course(course_slug, resolution, download_location, download_subtitle)

# Attach button click event to button
download_button.on_click(on_download_button_click)

# Display widgets and button
widgets.VBox([
    course_slug_widget,
    resolution_widget,
    download_location_widget,
    download_subtitle_widget,
    download_button
])


In [ ]:
#@title Upload directory to OneDrive using rclone

import os
import subprocess
import ipywidgets as widgets
from google.colab import files

# Install rclone
if not os.path.exists('/usr/bin/rclone'):
  print('Installing rclone...')
  !curl https://rclone.org/install.sh | sudo bash
  
# Prompt the user to upload rclone configuration file
print('Please upload the rclone configuration file.')
uploaded = files.upload()
config_file = next(iter(uploaded))
if config_file != 'rclone.conf':
  raise ValueError('Invalid file. Please upload the rclone configuration file.')

# Prompt the user for the directory to upload
print('Please enter the path of the directory to upload.')
upload_dir_widget = widgets.Text()
display(upload_dir_widget)

# Define callback function to run when upload button is clicked
def on_upload_button_click(b):
  # Get the selected directory
  upload_dir = upload_dir_widget.value
  # Upload the directory to OneDrive using rclone
  command = f'rclone copy "{upload_dir}" remote:onedrive --config "{config_file}" --progress'
  subprocess.call(command, shell=True)
  print('Upload complete.')

# Display the upload button
upload_button = widgets.Button(description='Upload')
upload_button.on_click(on_upload_button_click)
display(upload_button)


In [ ]:
#@title (Expermental Gdrive Upload)

import shutil
import os
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display

# Mount Google Drive
drive.mount('/content/drive')

# Create a button and a text box for the source and destination directories
source_widget = widgets.Text(description='Source directory:', value='/content')
destination_widget = widgets.Text(description='Destination directory:', value='/content/drive/MyDrive')

copy_button = widgets.Button(description='Copy')

progress_bar = widgets.FloatProgress(min=0, max=1.0)

def on_copy_button_click(b):
  # Get the source and destination directories
  source_dir = source_widget.value
  destination_dir = destination_widget.value
  
  # Get the number of files to be copied
  num_files = sum([len(files) for r, d, files in os.walk(source_dir)])
  
  # Update the progress bar for every file that is copied
  progress_bar.value = 0
  count = 0
  for root, dirs, files in os.walk(source_dir):
      for file in files:
          source_file = os.path.join(root, file)
          destination_file = os.path.join(destination_dir, os.path.relpath(source_file, source_dir))
          shutil.copy2(source_file, destination_file)
          count += 1
          progress_bar.value = count / num_files
  
  # Update the progress bar to indicate that the copy is complete
  progress_bar.value = 1

# Attach the function to the button click event
copy_button.on_click(on_copy_button_click)

# Display the widgets
display(source_widget)
display(destination_widget)
display(copy_button)
display(progress_bar)
